In [118]:
import pandas as pd
import numpy as np

# Read the data
id = "ff48df57-58c2-4eb4-9c12-e604c9651790"

Datasotre_API = f"https://ckan.indiadataportal.com/api/3/action/datastore_info?id={id}&include_unique=false"
Resource_API = f"https://ckan.indiadataportal.com/api/3/action/resource_show?id={id}"


df = pd.read_json(Datasotre_API)
data = df['result']['fields']

resourceData = pd.read_json(Resource_API)

In [119]:
primary_contains = ["id"]
temporal_contains = ["date","time","year","month","day","hour","minute","second"]
ordinal_dtypes_contains = ["int","float","double","numeric"]
ordinal_ignore = ["code"]

In [120]:
primary_col = []
temporal_cols = []
ordinal_cols = []
category_cols = []

for col in data:
    if col["schema"]["is_index"]:
        primary_col.append(col["id"])
    elif any(x in col["id"].lower() for x in temporal_contains):
        temporal_cols.append(col["id"])
    elif any(x in col["type"].lower() for x in ordinal_dtypes_contains) and not any(x in col["id"].lower() for x in ordinal_ignore):
        ordinal_cols.append(col["id"])
    else:
        category_cols.append(col["id"])

In [123]:
name = resourceData["result"]["name"].replace(" ","_")
yml_dict = {
    "cubes":
    [
        {
            "name" : name,
            "joins": [],
            "dimensions": [],
            "measures": [],
        }
    ]
}

for cols in ordinal_cols:
    yml_dict["cubes"][0]["measures"].append({
        "name": cols,
        "sql": "SUM("+cols+")",
        "type": "number"
    })

# add to inculude primary key in datamodeling

# yml_dict["cubes"][0]["dimensions"].append({
#     "name": primary_col[0],
#     "sql": primary_col[0],
#     "type": "string",
#     "primaryKey": True
# })


for cols in category_cols:
    yml_dict["cubes"][0]["dimensions"].append({
        "name": cols,
        "sql": cols,
        "type": "string"
    })

for cols in temporal_cols:
    yml_dict["cubes"][0]["dimensions"].append({
        "name": cols,
        "sql": cols,
        "type": "time"
    })

In [124]:
import yaml
class MyDumper(yaml.Dumper):
    def increase_indent(self, flow=False, indentless=False):
        return super(MyDumper, self).increase_indent(flow, False)

with open(name+".yml", 'w') as outfile:
    outfile.write(yaml.dump(yml_dict,Dumper=MyDumper,sort_keys=False))
    outfile.write("    sql_table: \"public.\\\""+id+"\\\"\"")
    outfile.write("\n")

In [ ]:
cube_name = name

In [18]:
import yaml

with open("cube_store_mapping.yml", 'r+') as file:
    documents = yaml.full_load(file)
    if not documents:
        documents = {}
        print("empty")
    print(documents)
    documents["33e3"] = [{"res_id":9330203},{"orignal_name":"nejnfjnejnfe"}]
    print(documents)
    file.seek(0)
    yaml.dump(documents, file)
    

{'Basic_Statistical_Return': [{'res_id': 'fd6eaa8e-ce59-4a10-8237-d6bfcd2f5358'}, {'orignal_name': 'Basic Statistical Return (BSR)- Credit by Scheduled Commercial Banks (SCBs)'}]}
{'Basic_Statistical_Return': [{'res_id': 'fd6eaa8e-ce59-4a10-8237-d6bfcd2f5358'}, {'orignal_name': 'Basic Statistical Return (BSR)- Credit by Scheduled Commercial Banks (SCBs)'}], '33e3': [{'res_id': 9330203}, {'orignal_name': 'nejnfjnejnfe'}]}
